# Trabajo práctico 1 : Analisis exploratorio del dataset Properatti

Grupo #11: Camila Coltriani, Irania Fuentes, Johnatan Fischelson, Luis Dartayet, Ornela Cevolli  

Para este trabajo utilizaremos un dataset de la inmobiliaría Properati construido con los datos de venta de propiedades en diferentes provincias de Argentina en el primer semestre del año 2017.

# Identificar el problema ¿o cambiar por objetivos?

El objetivo de este trabajo es realizar una limpieza del dataset properatti con la finalidad de obtener un dataset final con datos confiables que pueda ser utilizado en la generación de un modelo estadistico posterior.
Con base en esto se plantean los siguientes objetivos especificos:
 - Adquirir los datos: leer y conocer su estructura para determinar las herramientas apropiadas para su manipulación.

 - Parsear los datos: realizar el analisis exploratorio de los datos que permita verificar la existencia o no de relaciones entre variables, valores duplicados, valores faltantes, valores atípicos o valores erroneos que para validar o aumentar la confiabilidad de los datos.

 - Minar los datos: aplicar las herramientas de python para corregir datos erroneos o duplicados, completar/eliminar valores nulos.
 
 - Refinar los datos: eliminar variables redundantes o repetidas, crear nuevas variables y dar un formato limpio al dataset original.

# Adquirir y visualizar el dataset

In [ ]:
#librerías utilizadas para la adquisición de los datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import re
import seaborn as sns
import geopandas as gpd
import shapely.wkt
from decimal import Decimal
import re

Otros recursos utilizados
- dataset de id_geonames: ar_copy.csv
- dataset de barrios Argentina: barrios.csv 
- dataset de municipios Argentina del IGN: municipio.shp

In [ ]:
# Leemos y asignamos el dataset properatti.csv en una variable 
data = pd.read_csv("./properatti.csv", index_col=0)

In [ ]:
# Visualización de la forma y atributos del dataset 
print(data.shape)
print("El dataset está compuesto por:", data.shape[0], "filas y",data.shape[1],"columnas.")
data.sample(5) #

### Descripción de las columnas del dataset:

Los atributos o columas que incluye son:

● unmaded: 0: indice de filas

● property_type: tipo de inmueble en venta (casa, departamento, ph...)

● operation: tipo de operacion inmobiliaria para las propiedades 

● place_name: ubicacion del inmueble por ciudad/Partido o barrios

● place_with_parent_names: ubicacion agrupada del inmueble (Pais|Provincia|Partido o barrio)

● country_name: nombre del país donde ocurre la operacion inmobiliaría

● state_name: ubicacion del inmueble por provincia

● geonames_id: número de identificación en la base de datos GeoNames asociado a la ubicacion por coordenadas

● lat-lon: ubicacion de latitud y longitud concatenada

● lat  ●lon: ubicacion de latitud y longitud en columnas separadas

● price: precio del inmueble

● currency: divisa en la que está expresado el precio del inmueble

● price_aprox_local_currency: Precio aproximado en la moneda local del país de publicación

● surface_total_in_m2: superficie total m² del inmueble

● surface_covered_in_m2: Superficie cubierta en m²

● price_usd_per_m2: Precio en dolares por metro cuadrado (USD/m²: precio dólares / superficie)

● price_per_m2: Precio del metro cuadrado del inmueble

● floor: N° de piso (cuando corresponde)

● room: cantidad de habitaciones

● expenses: expensas (cuando corresponde)

● properati_url	: URL de la inmobiliaría Properati en la Web

● description: descripción del inmueble en la publicación Web

● title: título del inmueble en la publicación

● image_thumbnail: URL de un thumbnail de la primer foto en la Web

# Parsear los datos

## Analisis exploratorio general del dataset Properatti

In [ ]:
#Identificamos el tipo de dato de cada columna
data.dtypes
# El tipo de datos para variables cuantitativas discreta como floor y rooms deberia ser int, 
# posiblemente tengamos que realizar el cambio en su manipulación.

In [ ]:
# Realizamos una descripción estadística de todas las columnas ("include all también muestra las variables categóricas") 
# que resume la tendencia central, la dispersión y la forma de la distribución de un conjunto de datos
data.describe(include="all")

# Algunas interpretaciones/inferencias:
# operation y country_name tiene 1 solo dato:  Sell y Argentina, como ya sabíamos, el dataset son datos de venta en Argentina
# Existen cuatro tipos de propiedades en venta, la más frecuente es apartamento
# Placename tiene como dato más frecuente la ciudad de Cordoba y state_name tiene a Capital Federal
# lat-long hay datos repetidos o son los mismos edificios representados en un area determinada
# Existen valores máximos muy alejados del resto de datos en las columnas de superficie, floor y rooms, posibles outliers

In [ ]:
#Identificamos los valores únicos x columna
for columnas in data.columns:
    print("")
    print(f'Nombre:{columnas}')
    print(data[columnas].value_counts())

# de esta función sumamos información general sobre el data set:
# identificamos los tipos de inmueble en venta: apartamentos y casas concentran la mayoría de datos
# las divisas más utilizadas son el peso argentina y dólares, hay datos que podemos tomar como no representativos:
# el PEN: peso peruano y UYU: peso uruguayo ya que no pasan de dos registros en el dataset. 
# para floor y rooms hay que tratar los valores outliers

### Análisis de datos faltantes

In [ ]:
#Vemos la cantidad de datos nulos por variable
#creamos un nuevo dataframe con la suma de todos los registros nan por columna y el % que representan en forma decreciente
missing_data = data.isna().sum(axis=0)
missing_data_df = pd.DataFrame(missing_data, columns=['total_nan'])
missing_data_df['perc_%'] = (missing_data_df / data.shape[0]).round(2)*100
missing_data_df.sort_values(ascending=False, by='total_nan')

# price, currency, price_aprox, price_usd tienen la misma cantidad de nulos 20410
# hay que averiguar si price_per_m2 es la relación de price y surface_total, así podríamos completar nulos en price_per_m2
# podríamos completar place_name por el % bajo de nan
# el mayor % de nan se encuentra en floor, rooms y expensas


In [ ]:
# Analizamos por fila cantidad de datos faltantes
missing_by_row = (data.isna().sum(axis=1).value_counts() / data.shape[0] *100).sort_index() 
missing_by_row

In [ ]:
# Lo graficamos
sns.barplot(x=missing_by_row.index, y=missing_by_row.values) #grafico de barras
plt.title("Porcentaje de registros con cantidad de datos faltantes por filas")
plt.xlabel("Cantidad de datos faltantes")
plt.ylabel("Porcentaje de registros")
plt.show()

### Dispersión de datos

In [ ]:
# Vemos el porcentaje de datos unicos con respecto al total. 
# Nos indica que tan diferentes son los datos en cada variable.  
data_dispersion = data.apply(lambda x: x.unique().size)
data_dispersion_df = pd.DataFrame(data_dispersion, columns=['count'])
data_dispersion_df["perc"] = (data_dispersion / data.shape[0]).round(3)*100
data_dispersion_df.sort_values(ascending=True, by='count')
#vemos que operation y country_name tienen porcentaje cero ya que tienen un único registro.
#el aumento del porcentaje nos indica que tan diferentes son los datos dentro de la variable.

In [ ]:
#quisimos ver cuales son las variables con datos únicos menores a 100. 
print("Las variables con una cantidad de registros menores a 100 son:", )
for col in data.columns:
    if(data[col].nunique() < 100):
        print(col)
        print(data[col].unique())
        print()

### Analisis de correlacion entre columnas

In [ ]:
# Identificamos si existe una correlacion entre las diferentes variables numericas del dataset
corr = data.set_index("place_name").corr()
sm.graphics.plot_corr(corr, xnames=list(corr.columns))
plt.show()

# Hay una alta correlación entre price y price_per_m2, price_aprox_local_currency, price_aprox_usd 
## hay que ver el tipo de moneda de price seguramente comparta la misma que price_aprox_local_currency
# La relación entre price y price_per_m2 se puede deber a que se utiliza price para determinar el precio por m2 del inmueble
# la relación entre los variables precios y variables de superficies puede deberse que el precio sea una variable dependiente de la superficie 

## Exploración del dataset dividido en grupos de Properatti 

Para los siguientes pasos trabajaremos con el dataset dividido en tres grandes grupos:
-  Localización: que contiene las columnas relacionadas con la ubicación del inmueble y a su vez este dividido en dos subgrupos:
    - Ubicación: que contiene las columnas place_name, state_name, country_name y place_with_parent_names, es decir columnas con la localización política del inmueble.
    - Georeferenciada: que contiene las columnas geonames_id, lat y lon, es decir columnas con la localización geográfica del inmueble.
- Precio: que contiene las columnas relacionadas a los precios en distintas variantes. 
- Superficie: que contiene las columnas relacionadas con la superficie del inmueble

In [ ]:
# Definimos 4 grupos de variables para poder trabajar con ellas de forma mas sencilla
places = ['place_name','place_with_parent_names', 'country_name','state_name']
geolocation = ['geonames_id', 'lat-lon','lat','lon']
price = ['price','currency','price_aprox_local_currency','price_aprox_usd','price_usd_per_m2','price_per_m2']
surface = ['surface_total_in_m2','surface_covered_in_m2']

### Localización por ubicación política

Exploración de las variables relacionadas con la ubicación: Por Provincia, Ciudad/Barrio, el conjunto de ubicación 
- country_name
- state_name 
- place_name                        
- place_with_parent_names              

vamos a:

- Explorar las variables 
- Visualizar los nan de estas columnas
- Relacionar las columnas country_name, state_name, place_name con la concatenación de ubicación en place_with_parent_names para ver si efectivamente corresponden o hay datos mal cargados

In [ ]:
#Exploramos el conjunto total y las variables 
# Creamos un nuevo dataframe con las columnas que nos interesan
location_data = data[places].copy()
location_data.head()

In [ ]:
places_data = data[places].copy()   #asignamos el dataframe a la variable que utilizaremos en adelante
places_data.head()

In [ ]:
places_data.sample(10)

In [ ]:
#Vemos más a detalle por variable los registros totales, los registros únicos, el dato más frecuente y su frecuencia
#Vemos que state_name contiene más de las 23 provincias que debería tener Argentina:posiblemente estén datos diferentes a provincias
places_data[places].describe()

In [ ]:
#ubicación "State_name" ¿por provincia?
#vamos a ver un poco más cuales son los valores únicos de state_name que deberían relacionarse al nombre de las provincias
places_data["state_name"].value_counts() #Excludes NA values by default.

#efectivamente state_name si contiene las 23 provincias argentinas, sin embargo, divide a Buenos Aires en la capital federal
# y el gran buenos aires más la costa atlántica 
# observamos que los datos están concentrados en Buenos Aires (Cap fed, zona norte, zona sur, oeste, atlántica, interior),
#  Cordoba, Santa fe 

In [ ]:
# ubicación por place_name ¿ciudad-municipio o barrio?
places_data["place_name"].value_counts().head(20)
# NOtamos que existe una mexcla entre ciudades,municipios y barrios.
# Tigre es una ciudad al norte de la ciudad de Buenos Aires.
# Nordelta es una localidad urbana en el Partido de Tigre, Provincia de Buenos Aires 
## esta columna mezcla ciudades y barrios
# Capital Federal está como place_name 

In [ ]:
#nVemos algunos registros de place_with_parent_names
places_data["place_with_parent_names"].value_counts().head(10) 

# La variable representa el país/ "country_name", la provincia/ "state_name" (o division de la provincia), la ciudad principal/partido-municipio o barrio
# en el caso de capital federal"place_name"
# y un adicional que puede ser una localidad del partido (ejem: |Argentina|Bs.As. G.B.A. Zona Norte|Tigre|*Nordelta*|)
# los registros tienen entre 2  y 4 concatenaciones: 2 concatenaciones solo muestra hasta la ubicación por provincia, no brinda información relevante
### |Argentina|Capital Federal|  1297 no tiene nombre de barrio, desestimar o completar? 
### |Argentina|Córdoba|    2648no tiene nombre de barrio, desestimar o completar? 
# esta variable puede usarse para completar los datos faltantes de place_name 
# podemos chequear si el atributo "place_name" coincide con los datos aquí plasmados

In [ ]:
# Para state_name vemos: cuales son los datos de place_name más frecuente, su frecuencia y cuantos registros diferentes existen 
places_data.groupby(["state_name"])["place_name"].describe()

# La variable place_name está representado principalmente por una ciudad-municipio de la provincia o division de esta

## Verificar la calidad de los datos:


### Ubicación

Para verificar la calidad de los datos, principalmente, vamos a comparar las columnas que deberían tener la misma información y ver si coinciden o no. Por ejemplo, la columna "place_with_parent_names" debería contener dentro de su array la misma información que "place_name". Vamos a verificar si esto es cierto o no.

#### Corroborar si place_with_parent_names coincide con country_name, state_name, place_name 

In [ ]:
# Convertimos la columna place_with_parent_names en una lista usando el separador "|"
# Eliminamos el separador inicial y final de la lista
places_data['place_with_parent_names'] = places_data['place_with_parent_names'].apply(lambda x: x.lstrip("|").rstrip("|").split("|"))
places_data['place_with_parent_names'].head()

In [ ]:
# Función para chequear si los valores de la columna place_with_parent_names coinciden exactamente con 
# los valores de las columnas country_name, state_name, place_name
# Coincidir exactamente significa que la columna place_with_parent_names sólo tiene 3 elementos 
# y que los valores de los elementos coinciden en orden con los valores de las columnas country_name, state_name, place_name

def is_location_different(row):
    # la lista debería tener 3 elementos
    if len(row['place_with_parent_names']) != 3:
        return True
    if row['country_name'] == row['place_with_parent_names'][0] \
    and row['state_name'] == row['place_with_parent_names'][1] \
    and row['place_name'] == row['place_with_parent_names'][2]:
        return False
    else:
        return True

In [ ]:
# Creamos la mascara utilizando la función is_location_different
mask = places_data.apply(lambda x: is_location_different(x), axis=1)
print("place_with_parent_names difference with country_name, state_name and place_name:", places_data[mask].shape[0])
places_data[mask][places].sample(10)

*Conclusión: Existen 45220 registros que no coinciden exactamente.* 

#### Análisis de los registros que no coinciden exactamente en las columnas places
___

Veamos cuáles son los registros que no coinciden exactamente en las columnas places

In [ ]:
# Contamos la cantidad de valores de cada array de place_with_parent_names
destructured_places_with_parent_names_rows =  places_data['place_with_parent_names'].apply(lambda x: len(x)).value_counts()
destructured_places_with_parent_names_rows
# Este código nos devuelve la cantidad de filas que tenemos agrupadas en place_with_parent_names: va de dos a cinco especificaciones
# para la ubicación de un inmueble; hasta donde conocíamos, veníamos viendo hasta 3: País|Provincia|Ciudad o Barrio.
# veremos que descripción adicional nos brinda las concatenaciones >3.

In [ ]:
# Vemos el porcentaje para graficarlo luego
destructured_places_with_parent_names_rows_percent =  places_data['place_with_parent_names'].apply(lambda x: len(x)).value_counts(normalize=True) * 100
destructured_places_with_parent_names_rows_percent

In [ ]:
# Representamos la cantidad de grupo de datos contenidos en la variable place_with_parent_names
fig, ax = plt.subplots(figsize=(10,2))
sns.barplot(x=destructured_places_with_parent_names_rows_percent.values,y=destructured_places_with_parent_names_rows_percent.index, orient='h', ax=ax, ) 
ax.set_title('Cantidad de valores de cada array de place_with_parent_names') 
ax.set_xlabel('Porcentaje sobre el total') 
ax.set_ylabel('Valores de cada array')
plt.xlim(0,100)
plt.show()


De los pasos anteriores notamos: los registros que no coinciden exactamente son aquellas listas que no tienen 3 valores. De todas maneras tanto aquellos que tiene 2 como los que tienen 5 valores no tienen una cantidad significativa de registros. 

Revisamos que los valores que están en cada lista tengan su contraparte en las otras columnas aunque figuren en otro orden del array. Por ejemplo, si en place_with_parent_names figura "Argentina|Capital Federal|Palermo" y en place_name figura "Capital Federal", entonces el registro para este momento del análisis se considera válido.

In [ ]:
# country_name
print("null values:", places_data['country_name'].isna().sum())
mask = ~places_data.apply(lambda x: x['country_name'] in x['place_with_parent_names'], axis=1)
print("place_with_parent_names difference with country_name:", places_data[mask].shape[0])

In [ ]:
# state_name
print("null values:", places_data['state_name'].isna().sum())
mask = ~places_data.apply(lambda x: x['state_name'] in x['place_with_parent_names'], axis=1)
print("place_with_parent_names difference with state_name:", places_data[mask].shape[0])

In [ ]:
# place_name
print("null values:", places_data['place_name'].isna().sum())
mask = ~places_data.apply(lambda x: x['place_name'] in x['place_with_parent_names'], axis=1)
print("place_with_parent_names difference with place_name:", places_data[mask].shape[0])


*Conclusión: La información que figura en las columnas place_name, country_name y state_name se encuentra en la columna place_with_parent_names aunque no exactamente igual (parece existir información extra) excepto en la columna 'place_name' con sus 23 NaNs que ya hemos identificado previamente.*

A continuación analizaremos esas columnas dividiendo el trabajo en partes dependiendo de la cantidad de elementos que contenga el array de la columna "place_with_parent_names".

#### Análisis de registros de place_with_parent_names con 3 valores
___

In [ ]:
# Revisamos los place_with_parent_names con tres elementos
mask = places_data['place_with_parent_names'].apply(lambda x: len(x) == 3)
places_data_3_elements = places_data[mask].copy()
places_data_3_elements.shape

In [ ]:
# Corroboramos que los valores de country_name, state_name y place_name coinciden con los valores de place_with_parent_names
print("country_name difference with place_with_parent_names[0]:", places_data_3_elements[places_data_3_elements['country_name'] != places_data_3_elements['place_with_parent_names'].apply(lambda x: x[0])].shape[0])
print("state_name difference with place_with_parent_names[1]:", places_data_3_elements[places_data_3_elements['state_name'] != places_data_3_elements['place_with_parent_names'].apply(lambda x: x[1])].shape[0])
print("place_name difference with place_with_parent_names[2]:", places_data_3_elements[places_data_3_elements['place_name'] != places_data_3_elements['place_with_parent_names'].apply(lambda x: x[2])].shape[0])

In [ ]:
# Vemos los datos correspondientes a los valores nulos de la columna place_name
mask = ~places_data.apply(lambda x: x['place_name'] in x['place_with_parent_names'], axis=1)
places_data_place_name_nan = places_data[mask].copy()
places_data_place_name_nan.head()

In [ ]:
#Extraemos el dato en la posicion que corresponde a Tigre y lo contabilizamos 
places_data_place_name_nan['place_with_parent_names'].apply(lambda x: x[2]).value_counts()

*Conclusión: Los registros que no tienen valor en place_name son los que no coinciden, todos del municipio de Tigre. El resto de los valores son exactamente iguales*  





#### Análisis de registros de place_with_parent_names con 4 valores
___

In [ ]:
# Revisamos los place_with_parent_names con cuatro elementos
mask = places_data['place_with_parent_names'].apply(lambda x: len(x) == 4)
places_data_4_elements = places_data[mask].copy()
places_data_4_elements.shape


In [ ]:
# Contamos los valores de la columna place_with_parent_names por el segundo elemento (descartamos el primero
# porque sabemos que siempre es Argentina)
places_data_4_elements['place_with_parent_names'].apply(lambda x: x[1]).value_counts()


In [ ]:
# Confirmamos que los valores de la columna place_with_parent_names[1] coinciden con los valores de la columna state_name
mask = (places_data_4_elements['place_with_parent_names'].apply(lambda x: x[1]) != places_data_4_elements['state_name'])
print("place_with_parent_names[1] difference with state_name:", places_data_4_elements[mask].shape[0])


In [ ]:
# Confirmamos que los valores de la columna place_with_parent_names[2] coinciden con los valores de la columna place_name
mask = places_data_4_elements['place_with_parent_names'].apply(lambda x: x[2]) != places_data_4_elements['place_name']
print("place_with_parent_names[2] difference with place_name:", places_data_4_elements[mask].shape[0])
print("place_with_parent_names[2] proportional difference with place_name:", places_data_4_elements[mask].shape[0] / places_data_4_elements.shape[0])
places_data_4_elements_non_matching = places_data_4_elements[mask].copy()
places_data_4_elements_non_matching[places].sample(10)

In [ ]:
# No coinciden en 29842 registros pero coinciden en el resto. 
# Confirmamos que los que coinciden son iguales al valor[3] en la cadena completa
mask = places_data_4_elements_non_matching['place_with_parent_names'].apply(lambda x: x[2]) == places_data_4_elements_non_matching['place_with_parent_names'].apply(lambda x: x[3])
places_data_4_elements_non_matching[mask].shape[0]/ places_data_4_elements_non_matching.shape[0]

In [ ]:
# Confirmamos que los valores de la columna place_with_parent_names[3] coinciden con los valores de la columna place_name
mask = (places_data_4_elements['place_with_parent_names'].apply(lambda x: x[3]) != places_data_4_elements['place_name'])
print("place_with_parent_names[3] difference with place_name:", places_data_4_elements[mask].shape[0])
anti_mask = ~mask # Los que coinciden
places_data_4_elements[anti_mask][places].sample(10)

*Conclusión: De los 39869 registros con 4 valores, 29842 toman el valor de place_name del 4to valor de la lista. Los 10027 restantes toman el 3er valor.* que es siempre un barrio/localidad dentro del municipio: ejm: Glew localidad del municipio Almiramte Brown de zona sur del Gran Buenos Aires.

- place_name asignado a 3er valor de "place_with_parent_names": 10027
- place_name asignado a 4to valor de "place_with_parent_names": 29842

#### Análisis de registros de place_with_parent_names con 2 valores
___

In [ ]:
# Reviso los place_with_parent_names con dos elementos
mask = places_data['place_with_parent_names'].apply(lambda x: len(x) == 2)
places_data_2_elements = places_data[mask].copy()
places_data_2_elements['place_with_parent_names'].value_counts()

In [ ]:
# Reviso los place_with_parent_names de Córdoba y Capital Federal por contener el maypr numero de registros
mask = places_data_2_elements['place_with_parent_names'].apply(lambda x: x[1] == 'Córdoba' or x[1] == 'Capital Federal')
places_data_2_elements[mask].sample(10)

In [ ]:
# Pareciera ser que todos los lugares que tienen dos elementos en place_with_parent_names repiten el nombre del estado en place_name
# Corroboramos esto
mask = places_data_2_elements['place_name'] != places_data_2_elements['state_name']
places_data_2_elements[mask].shape


*Conclusión: En los 4780 registros de place_with_parent_names que tienen 2 valores se utilizó state_name como place_name.* 
*Es decir, la ciudad principal de la provincia.*

#### Análisis de registros de place_with_parent_names con 5 valores
___

In [ ]:
# Revisamos los place_with_parent_names con 5 elementos
mask = places_data['place_with_parent_names'].apply(lambda x: len(x) == 5)
places_data_5_elements = places_data[mask].copy()
print(places_data_5_elements['place_with_parent_names'].shape)
places_data_5_elements.sample(5)

In [ ]:
# Revisamos si alguno de los valores no corresponde a Nordelta
places_data_5_elements['place_with_parent_names'].apply(lambda x: x[3] != 'Nordelta').sum()

In [ ]:
# Corroboramos que los valores de la columna place_with_parent_names[2] coinciden con los valores de la columna place_name
mask = (places_data_5_elements['place_with_parent_names'].apply(lambda x: x[2]) != places_data_5_elements['place_name'])
print("place_with_parent_names[2] difference with place_name:", places_data_5_elements[mask].shape[0])
print("place_with_parent_names[2] proportional difference with place_name:", places_data_5_elements[mask].shape[0] / places_data_5_elements.shape[0])


In [ ]:
# Corroboramos que los valores de la columna place_with_parent_names[3] coinciden con los valores de la columna place_name
mask = (places_data_5_elements['place_with_parent_names'].apply(lambda x: x[3]) != places_data_5_elements['place_name'])
print("place_with_parent_names[3] difference with place_name:", places_data_5_elements[mask].shape[0])
print("place_with_parent_names[3] proportional difference with place_name:", places_data_5_elements[mask].shape[0] / places_data_5_elements.shape[0])


In [ ]:
# Corroboramos que los valores de la columna place_with_parent_names[4] coinciden con los valores de la columna place_name
mask = (places_data_5_elements['place_with_parent_names'].apply(lambda x: x[4]) != places_data_5_elements['place_name'])
print("place_with_parent_names[4] difference with place_name:", places_data_5_elements[mask].shape[0])
print("place_with_parent_names[4] proportional difference with place_name:", places_data_5_elements[mask].shape[0] / places_data_5_elements.shape[0])

*Conclusión: Los 548 registros de place_with_parent_names que tienen 5 valores son de Nordelta y el último valor se refiere al Barrio. place_name toma los valores del barrio*


> ##### Conclusiones finales: 
> - Existen como máximo 5 tipos de registros en place_with_parent_names: País, Provincia, Municipio, Ciudad, Barrio
> - País y Provincia son iguales a country_name y state_name
> - Place_name se le asigna el valor de municipio: 76000 + 10027 = 86027
> - Place_name se le asigna el valor de ciudad: 29842 (en  estos casos existe el valor municipio también )
> - Place_name se le asigna el valor de barrio: 548 (en estos casos existe el valor municipio y ciudad también)
> - Place_name se le asigna el valor de state_name: 4780 (en estos casos no existe el valor municipio y ciudad)
> - Place_name se le asigna el valor de NaN: 23 (en estos casos existe el valor municipio y ciudad)
> ___
> Por lo tanto se puede concluir que para dar mayor consistencia es posible imputar place_name con el valor del municipio en las mayoría de los casos. Confrontar con geoNamesId.
> ___








### Geolocalización

In [ ]:
# Creamos un nuevo dataframe con las columnas que nos interesan
geo_location_data = data[ geolocation].copy()
geo_location_data.head()

Analizamos  lat-lon para ver si habían registros duplicados considerando que si existe una misma coordenadas son las misma propiedad.

In [ ]:
## Creamos una copia de data en la que podamos borrar los nan y agrupamos por lat y lon, contamos sus valores y
# le decimos que nos devuelva cuantos registros hay mayores a 1 en forma descendente para todo el dataset.
data_copy = data.copy()
data_copy.dropna(subset=['lat-lon'], inplace=True)
data_copy_group = data_copy.groupby('lat-lon').count()
data_copy_group[data_copy_group['place_name'] > 1].sort_values(by='operation', ascending=False)

#llama la atención que existan coordenadas repetidas, podríamos inferir que las coordenadas no son especificas de la ubicación del
# inmueble, sino de un punto de referencia  

In [ ]:
## seguimos chequeando si si existe algún dato duplicado en lat-lon
print(data.duplicated().any())
print(data['lat-lon'].duplicated().any())
print(data_copy.shape)

# no existen columnas duplicadas, pero si existen datos de lat-lon iguales en un total de 69670 filas 

In [ ]:
#Quisimos ver los datos completos que tienen como coordenadas los 312 casos que vimos anteriormente
data_copy[data_copy['lat-lon'] == '-34.4026444,-58.6684776']

#vemos que los datos repetidos de lat y lon no se deben al mismo inmueble

In [ ]:
geo_location_data.sample(10)


No es posible determinar si es la misma propiedad porque lat-lon se refiere muchas veces a la ubicación aproximada. 

In [ ]:
# Vemos el porcentaje de valores nulos por columna
geo_location_data.isnull().sum()/geo_location_data.shape[0] * 100


In [ ]:
# Visualizamos en el mapa de SurAmerica si los datos de lat y lon pertenecen efectivamente a Argentina
# convertimos el dataframe a geodataframe
geo_location_data_gdf = gpd.GeoDataFrame(geo_location_data, geometry=gpd.points_from_xy(geo_location_data.lon, geo_location_data.lat))

# Ubicamos los puntos en el mapa
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
latin_america = world[world['continent'] == 'South America']
fig, ax = plt.subplots(figsize=(5,10))
latin_america.plot(ax=ax, alpha=0.4, color='grey', edgecolor='black')
geo_location_data_gdf.plot(ax=ax, markersize=0.5, color='red')

In [ ]:
# Existe una propiedad que se encuentra fuera de la Argentina (en Colombia). 
# La buscamos en el dataframe original
# buscamos el polígono de Colombia
geo_colombia = world[world['name'] == 'Colombia']

# Buscamos la propiedad que se encuentra en Colombia
geo_outlier_index = geo_location_data_gdf[geo_location_data_gdf.within(geo_colombia['geometry'].iloc[0])].index

# La buscamos en el dataframe original
data.iloc[geo_outlier_index]


Es una propiedad súper interesante, por latitud y longitud está en Colombia, pero por el nombre de la provincia está en Argentina, la descripción habla de Armenia y el título del el Barrio el Limonar.

En una rápida búsqueda por internet encontramos que la propiedad se encuentra en el barrio El Limonar de Armenia, Colombia.

La descartaremos sin dudas.

______

#### Corroborar si lat-lon coincide con las columnas lat y lon

In [ ]:
# Corroboremos que los valores nulos en lat-lon son los mismos que en lat y lon
geo_location_data[geo_location_data['lat-lon'].isnull()]['lat'].isnull().sum() == geo_location_data[geo_location_data['lat-lon'].isnull()]['lon'].isnull().sum()

In [ ]:
# Dropeamos los valores nulos de lat-lon
geo_location_data.dropna(subset=['lat-lon'], inplace=True)
geo_location_data.isnull().sum()

In [ ]:
# Convertimos la columna lat-lon en dos columnas nuevas
geo_location_data['lat_alt'] = geo_location_data['lat-lon'].apply(lambda x: x.split(',')[0])
geo_location_data['lon_alt'] = geo_location_data['lat-lon'].apply(lambda x: x.split(',')[1])
geo_location_data.head()


In [ ]:
# Vemos los tipos de datos
geo_location_data.dtypes

In [ ]:
# convertimos las nuevas columnas a float
geo_location_data['lat_alt'] = geo_location_data['lat_alt'].astype(float)
geo_location_data['lon_alt'] = geo_location_data['lon_alt'].astype(float)
geo_location_data.dtypes

In [ ]:
# Comprobamos que los valores de las nuevas columnas son iguales a los de las columnas originales
print("Diferencia entre lat:",(geo_location_data['lat_alt'] != geo_location_data['lat']).sum())
print("Diferencia entre lon:",(geo_location_data['lon_alt'] != geo_location_data['lon']).sum())
# En proporción
print("Diferencia en proporción de lat",(geo_location_data['lat_alt'] != geo_location_data['lat']).sum()/ geo_location_data.shape[0])
print("Diferencia en proporción de lon",(geo_location_data['lon_alt'] != geo_location_data['lon']).sum()/ geo_location_data.shape[0])


Hay diferencias en 21% los casos. Veamos si es una diferencia significativa

Revisamos el margen de diferencia redondeando progresivamente.

In [ ]:
Decimal(geo_location_data['lon'][0]).as_tuple().exponent

In [ ]:
# Reviso la cantidad de decimales que tienen los valores de lat y lon
geo_location_data['lat'].apply(lambda x: Decimal(x).as_tuple().exponent).value_counts()

In [ ]:
geo_location_data['lon'].apply(lambda x: Decimal(x).as_tuple().exponent).value_counts()

In [ ]:
# Vamos redondeando progresivamente los valores de lat y lon
lat_decimal_diff = []
for i in range(47,0,-1):
    margin = (geo_location_data['lat_alt'].round(i) != geo_location_data['lat'].round(i)).sum()/ geo_location_data.shape[0]
    lat_decimal_diff.insert(0,margin)
    # print('Margen de diferencia de',i,'decimales en latitud:', margin)
# Buscamos dónde el margen de diferencia es menor al 1%
    if margin <= 0.01:
        print('Margen de diferencia de',i,'decimales en latitud:', margin)
   
print("////////////////////////////////////")   

lon_decimal_diff = []
for i in range(47,0,-1):
    margin = (geo_location_data['lon_alt'].round(i) != geo_location_data['lon'].round(i)).sum()/ geo_location_data.shape[0]
    lon_decimal_diff.insert(0,margin)
    # print('Margen de diferencia de',i,'decimales en longitud:', margin)
# Buscamos dónde el margen de diferencia es menor al 1%
    if margin <= 0.01:
        print('Margen de diferencia de',i,'decimales en latitud:', margin)




In [ ]:
# Lo graficamos
plt.figure(figsize=(10,5))
plt.plot(lat_decimal_diff, label='latitud')
plt.plot(lon_decimal_diff, label='longitud')
plt.ylabel('Margen de diferencia')
plt.xlabel('Cantidad de decimales')
plt.legend()


In [ ]:
geo_location_data_gdf = gpd.GeoDataFrame(geo_location_data, geometry=gpd.points_from_xy(geo_location_data.lon, geo_location_data.lat))
geo_location_data_gdf.head()


> Como se puede ver, las diferencias entre lat-lon y lat y lon comienzan a partir de los 13 decimales por lo que podemos descartarla como una diferencia significativa.
> 
> Por otro lado, también hemos comprobado que los valores de lat lon provienen de convertir la variable a geometry y obtener de ahí lat y lon
>
> Entonces podemos concluir que la variable lat-lon es redundante y podemos eliminarla.
> ____ 

#### Análisis de propiedades con igual latitud y longitud con otras propiedades

Vamos a buscar si hay muchos puntos iguales para refutar la hipótesis de que se utilizó la misma ubicación para varias propiedades

In [ ]:
# Primero veamos una visualización de los datos de Capital Federal para ver si parece haber una gran concentración 
# de propiedades en algún lugar específico o están dispersas por toda la ciudad.
geo_location_data_gdf_capital = geo_location_data_gdf[data['state_name']=='Capital Federal' ]
geo_location_data_gdf_capital['geometry']

In [ ]:
# Ubicamos los puntos en el mapa
df_barrios_capital = pd.read_csv('./data/barrios.csv', sep=',', encoding='latin-1')
import shapely.wkt

df_barrios_capital["WKT"] = df_barrios_capital["WKT"].apply(shapely.wkt.loads) 
df_barrios_capital = gpd.GeoDataFrame(df_barrios_capital, geometry='WKT')


In [ ]:
fig, ax = plt.subplots(figsize=(10,20))

geo_location_data_gdf_capital.plot(ax=ax, markersize=10, color='red', alpha=0.1) 
df_barrios_capital.plot(ax=ax, alpha=0.4, color='grey', edgecolor='black')
plt.xlim(-58.55,-58.350)
plt.ylim(-34.705,-34.525) 
plt.show()

In [ ]:
# Ahora veamos en todo el dataset cuántas propiedades comparten la misma ubicación.
different_locations = geo_location_data_gdf['geometry'].value_counts()
print('Cantidad de propiedades por punto:', different_locations) 

In [ ]:
prop_qty_per_point = []
for i in range(1,different_locations[0]+1):
    (different_locations == i).sum() / different_locations.shape[0] * 100
    prop_qty_per_point.append((different_locations == i).sum() / different_locations.shape[0] * 100)
prop_qty_per_point = pd.DataFrame(prop_qty_per_point, columns=['Porcentaje'], index=range(1,different_locations[0]+1))
prop_qty_per_point.reset_index(inplace=True)
prop_qty_per_point.rename(columns={'index':'Cantidad de propiedades por punto'}, inplace=True)
prop_qty_per_point.head(10)

In [ ]:
# Lo graficamos
plt.figure(figsize=(10,5))
plt.hist(different_locations, bins=different_locations[0]+1)
plt.ylabel('Cantidad de puntos')
plt.xlabel('Cantidad de propiedades por punto')
plt.show()



Veamos dónde se encuentran las propiedades que más tienen lat y lon iguales

In [ ]:
most_shared_point_props = geo_location_data_gdf[geo_location_data_gdf['geometry'].isin(different_locations.index[0:11])]
 

In [ ]:
fig, ax = plt.subplots(figsize=(5,10))
latin_america.plot(ax=ax, alpha=0.4, color='grey', edgecolor='black')
most_shared_point_props.plot(ax=ax, markersize=0.5, color='red')

In [ ]:
# Parece que están todas en la Argentina
# Descargamos un mapa de la Argentina por municipios para averiguar qué municipios son los que tienen más propiedades en el mismo punto.
# https://www.ign.gob.ar/NuestrasActividades/InformacionGeoespacial/CapasSIG
municipios_geo = gpd.read_file('./data/municipio/municipio.shp')
municipios_geo.head()

In [ ]:
fig, ax = plt.subplots(figsize=(5,10))
municipios_geo.plot(ax=ax, alpha=0.4, color='grey', edgecolor='black')
plt.show()

In [ ]:
# Veamos en qué municipios están las propiedades que comparten el mismo punto.
# convertimos a coordenadas geográficas para poder hacer el join con el geodataframe de las propiedades.
most_shared_point_props.crs = "EPSG:4326"
most_shared_point_props = most_shared_point_props.to_crs(epsg=4326) 

In [ ]:
# Hacemos el join con el geodataframe de los municipios.
most_shared_point_props_municipios = gpd.sjoin(most_shared_point_props, municipios_geo, how="inner")
most_shared_point_props_municipios.head()

In [ ]:
most_shared_point_props_municipios['fna'].value_counts()

> Podemos concluir que la gran mayoría de los puntos de lat y lon son únicos, por lo que descartamos que sean aproximaciones.
>  
> Una excepción notable son Tigre y Capital Federal
> 
> ____

#### Corroborar si lat-lon coincide con geonames_id

In [ ]:
#creamos una copia de las variables de geolocación
geo_location_data = data[ geolocation].copy()

In [ ]:
# Corroboramos que todos los nulos de latitud y longitud también son nulos en la columna geonames_id.
lat_nulls = geo_location_data[geo_location_data['lat'].isnull()]
lon_nulls = geo_location_data[geo_location_data['lon'].isnull()]
geonames_null = geo_location_data[geo_location_data['geonames_id'].isnull()]
print('Es lat null = lon null?: ', lat_nulls.equals(lon_nulls))  
print('Es lat null = geonames null?: ', lat_nulls.equals(geonames_null))  
print('Es lon null = geonames null?: ', lon_nulls.equals(geonames_null))  


In [ ]:
# Vemos cuantas columnas tienen nulos en lat (o lon, son iguales en cuanto a nulos) y no en geonames_id.
lat_nulls = geo_location_data[geo_location_data['lat'].isnull()]
lat_nulls_geonames_not_null = lat_nulls[lat_nulls['geonames_id'].notnull()]
lat_nulls_geonames_not_null.shape[0] 

In [ ]:
# Vemos cuantas columnas tienen nulos en geonames_id y no en lat (o lon, son iguales en cuanto a nulos).
geonames_null = geo_location_data[geo_location_data['geonames_id'].isnull()]
geonames_null_lat_not_null = geonames_null[geonames_null['lat'].notnull()]
geonames_null_lat_not_null.shape[0]

> Conclusión: Podemos imputar lat y lon a partir de geonames_id en 43365 casos. 
> 
> Podríamos imputar 10532 casos en geonames pero con una sóla variable de geolocalización es suficiente y usaremos lat-lon para crear geometry.
>  
> ______

##### Verificamos si es posible importar la información de lat y lon desde geonames_id oficial 

In [ ]:
# leamos y cargamos el dataset con los geonames id de Argentina
geonames = pd.read_csv("./data/ar_copy.csv", sep='\t', header=None)

In [ ]:
# como no hay nombres en la columnas, para mejorar la extracción renombro las que me interesan
geonames.rename({0: 'geoname_oficial', 4:"lat_oficial", 5:"lon_oficial"}, axis=1, inplace=True)
geonames.head(4)

In [ ]:
# ahora buscamos los datos de lat y lon desde el geoname_oficial usando como clave el id geoname (geoname_oficial) y el dato de
# latitud y longitud asociada

#Creamos un diccionario vacío para ubicar la Latitud
lat_dict = {}
#Creamos una tupla con los pares de key y value: usando un iterador de tuplas zip donde el primer
# elemento de cada iterador pasado se empareja con el primero del segundo y asi sucesivamente
geoname_lat = zip(geonames['geoname_oficial'], geonames['lat_oficial'])

#completamos el diccionario con un for
for geoname, lat_oficial in geoname_lat:
    lat_dict[geoname] = lat_oficial

In [ ]:
#Creamos un diccionario para ubicar la Longitud
lon_dict = {}

#Creamos una tupla con los pares de key y value: usando un iterador de tuplas zip donde el primer
# elemento de cada iterador pasado se empareja con el primero del segundo y asi sucesivamente
geoname_lon = zip(geonames['geoname_oficial'], geonames['lon_oficial'])

#completamos el diccionario
for geoname, lon_oficial in geoname_lon:
    lon_dict[geoname] = lon_oficial

In [ ]:
#Agregamos dos nuevas columnas a nuestro dataset de geo_location con lat y lon mapeados del diccionario, para lo cual
#usando el geoname_id de nuestro dataset le indicamos que mapee si es el mismo entonces extraiga el valor de lat y lon
# a las nuevas columnas
geo_location_data["lat_geoname"] = geo_location_data['geonames_id'].map(lat_dict)
geo_location_data["lon_geoname"] = geo_location_data['geonames_id'].map(lon_dict)

In [ ]:
# Veamos cuantos valores con geonames_id no nulos no tienen latitud y longitud.
missing_georeference_from_geonames = geo_location_data[geo_location_data['lat_geoname'].isnull()]
missing_georeference_from_geonames.shape

In [ ]:
# Filtramos los que tienen geonames_id pero no latitud y longitud heredada por geonames.
missing_geo_names_location_data = geo_location_data[geo_location_data['lat_geoname'].isnull() & geo_location_data['geonames_id'].notnull()]
print(missing_geo_names_location_data.shape)
missing_geo_names_location_data.head()

In [ ]:
missing_geo_names_location_data['geonames_id'].value_counts() # Es Palermo, Capital Federal

*Los registros con Geonames_id 3435548 (que corresponden a Palermo) ya tienen lat y lon por lo no nos importa el lat y lon de geonames*

In [ ]:
# vemos la cantidad de registros nulos del data geo_location con los nuevos datos de lat y lon agregados
print(geo_location_data.isna().sum())
# Nos damos cuenta que siguen existiendo registros de latitud y longitud que no se pudieron mapear del dataset con id_geonames oficial.

In [ ]:
geo_location_data[geo_location_data['lat'].isnull() & geo_location_data['lat_geoname'].notnull()]

In [ ]:
#si usaramos los datos del dataset ar_copy pudieramos imputar el 37,19% de los datos faltantes de lat y lon en nuestro dataset
(geo_location_data["lat_geoname"].isna().sum() / geo_location_data["lat"].isna().sum() *100).round(2)

##### Comparar lat-lon con lat-lon extraído de geonames_id

Esto no se puede hacer porque sólo se extrajeron los datos de geonames_id que no tenían lat-lon

In [ ]:
# comparo los datos completados con los datos originales para ver si hay diferencias
# remuevo los nan de los datos originales (porque seguro va a haber diferencias en esos datos)
geo_location_data_geonames_vs_latlon = geo_location_data.dropna(subset=['lat', 'lon','geonames_id', 'lat_geoname', "lon_geoname" ])
geo_location_data_geonames_vs_latlon.isna().sum()  

In [ ]:
geo_location_data_geonames_vs_latlon.shape 

In [ ]:
# comparamos las filas en común que no tienen nulos
geo_location_data_geonames_vs_latlon['lat'] == geo_location_data_geonames_vs_latlon['lat_geoname']
geo_location_data_geonames_vs_latlon[geo_location_data_geonames_vs_latlon['lat'] != geo_location_data_geonames_vs_latlon['lat_geoname']]

In [ ]:
# Ubicamos los puntos en el mapa

# Estos df ya fueron creado y están puestos como referencia
# geo_location_data_gdf = gpd.GeoDataFrame(geo_location_data, geometry=gpd.points_from_xy(geo_location_data.lon, geo_location_data.lat))
# geo_location_data_gdf_capital = geo_location_data_gdf[data['state_name']=='Capital Federal']


geo_location_data_gdf_geonames = gpd.GeoDataFrame(geo_location_data_geonames_vs_latlon, geometry=gpd.points_from_xy(geo_location_data_geonames_vs_latlon.lon_geoname, geo_location_data_geonames_vs_latlon.lat_geoname))
df_barrios_capital = pd.read_csv('./data/barrios.csv', sep=',', encoding='latin-1')


df_barrios_capital["WKT"] = df_barrios_capital["WKT"].apply(shapely.wkt.loads) 
df_barrios_capital = gpd.GeoDataFrame(df_barrios_capital, geometry='WKT')

fig, ax = plt.subplots(figsize=(10,20))

geo_location_data_gdf_capital.plot(ax=ax, markersize=0.5, color='red', alpha=0.1) 
geo_location_data_gdf_geonames.plot(ax=ax, markersize=10, color='blue', alpha=0.1)
df_barrios_capital.plot(ax=ax, alpha=0.4, color='grey', edgecolor='black')
plt.xlim(-58.55,-58.350)
plt.ylim(-34.705,-34.525) 
plt.show()

*Conclusión: Geonames_id marca el centro de cada uno de los barrios y no la latitud y longitud exacta de la propiedad. Por lo que deberíamos imputar lat y lon a partir de geonames_id con mucho cuidado.*

#### Corroborar si lat-lon coincide con place_name

In [ ]:
municipios_geo.head()

In [ ]:
municipios_geo_nam_geo = municipios_geo.copy()

In [ ]:
# Reemplazamos ciudades por municipios
places_data_4_elements_with_mun_as_place_name = places_data_4_elements.copy()
places_data_4_elements_with_mun_as_place_name['place_name'] = places_data_4_elements['place_with_parent_names'].apply(lambda x: x[2])
print(places_data_4_elements_with_mun_as_place_name.shape)
places_data_4_elements_with_mun_as_place_name.head()

In [ ]:
# Unimos los nuevos municipios asignados con los otro municipios
places_data_mun = pd.concat([places_data_4_elements_with_mun_as_place_name, places_data_3_elements], axis=0) 
print(places_data_mun.shape) 
places_data_mun.head() 

In [ ]:
# Unimos los municipios con su georeferencia
places_data_mun_geo = places_data_mun.join(geo_location_data)
print(places_data_mun_geo.shape)
places_data_mun_geo.head()

In [ ]:
# Lo convertimos en geo data frame
places_data_mun_geo_gdf = gpd.GeoDataFrame(places_data_mun_geo, geometry=gpd.points_from_xy(places_data_mun_geo.lon, places_data_mun_geo.lat))
places_data_mun_geo_gdf.head()

In [ ]:
# limpiamos los datos que no tienen georeferencia
places_data_mun_geo_gdf.dropna(subset=['lat', 'lon'], inplace=True)
places_data_mun_geo_gdf = places_data_mun_geo_gdf[['place_name', 'geometry','state_name']]
print(places_data_mun_geo_gdf.shape)
places_data_mun_geo_gdf.head()

In [ ]:
# convertimos a coordenadas geográficas para poder hacer el join con el geodataframe de las propiedades.
places_data_mun_geo_gdf.crs = "EPSG:4326" 
places_data_mun_geo_gdf = places_data_mun_geo_gdf.to_crs("EPSG:4326")

In [ ]:
# unimos los municipios con su georeferencia con los municipios con su georeferencia de places_data_mun_geo_gdf
common_municipios = gpd.sjoin(municipios_geo_nam_geo, places_data_mun_geo_gdf, how="inner")
common_municipios = common_municipios[['nam','fna', 'place_name','state_name', 'geometry']]
print(common_municipios.shape)
common_municipios.head()

In [ ]:
# Chequeamos que nam y place_name sean iguales
(common_municipios['nam'] == common_municipios['place_name']).sum() == common_municipios.shape[0] 


In [ ]:
# Vemos que municipios que coinciden con place_name
common_municipios[common_municipios['nam'] == common_municipios['place_name']]

In [ ]:
# Vemos que municipios que no coinciden con place_name
common_municipios[common_municipios['nam'] != common_municipios['place_name']]

In [ ]:
# Considerando que el patrón Comuna n es de Capital Federal, eliminamos las filas que tenga ese patrón y el state_name sea Capital Federal
regex_comuna = r'Comuna \d+'
common_municipios = common_municipios[ ~((common_municipios['nam'].str.contains(regex_comuna)) & (common_municipios['state_name'] == 'Capital Federal'))]
print(common_municipios.shape)
uncommon_municipios = common_municipios[common_municipios['nam'] != common_municipios['place_name']]
print(uncommon_municipios.shape)
uncommon_municipios.head()

In [ ]:
print("Municipios diferentes: ", uncommon_municipios['nam'].nunique())
uncommon_municipios['place_name'].value_counts().head()

In [ ]:
uncommon_municipios_mar_del_plata = uncommon_municipios[uncommon_municipios['place_name'] == 'Mar del Plata'] 
print(uncommon_municipios_mar_del_plata.shape) 
uncommon_municipios_mar_del_plata.head()

In [ ]:
# Mar del Plata corresponde al municipio de General Pueyrredón, por lo que también lo eliminamos.
common_municipios = common_municipios[ ~((common_municipios['nam'] == 'General Pueyrredón') & (common_municipios['place_name'] == 'Mar del Plata'))]
print(common_municipios.shape)
uncommon_municipios = common_municipios[common_municipios['nam'] != common_municipios['place_name']]
print(uncommon_municipios.shape)
uncommon_municipios.head()

In [ ]:
uncommon_municipios.groupby([ 'nam','place_name']).count().sort_values(by='state_name', ascending=False).head(10)

*Conclusión: Los datos de lat-lon coinciden con place_name.*

Casos notables: 
- Capital Federal: según el df del IGN son comunas y según el original son barrios. Se asume que son correctos.
- Mar del Plata: Mar del Plata es la ciudad y General Pueyrredón el municipio. Si se utiliza el municipio como place_name lo correcto es imputarlo. # https://www.argentina.gob.ar/buenosaires/municipios

#### Analizar si es posible imputar place_name de los registros deplaces_data_2_elements(donde solo hay país y provincia)

In [ ]:
# Unimos los municipios con su georeferencia
places_data_2_elements_geo = places_data_2_elements.join(geo_location_data)  
print(places_data_2_elements_geo.shape)
places_data_2_elements_geo.head()

In [ ]:
# Vemos si existe geonames_id en alguna fila en places_data_2_elements_geo
places_data_2_elements_geo['geonames_id'].isnull().sum() == places_data_2_elements_geo.shape[0]  

In [ ]:
# Vemos si existe lat y lon en alguna fila en places_data_2_elements_geo
places_data_2_elements_geo['lat'].isnull().sum() == places_data_2_elements_geo.shape[0]  and places_data_2_elements_geo['lon'].isnull().sum() == places_data_2_elements_geo.shape[0]

*Conclusión: No es posible imputar place_name en los registros donde place name es una provincia a partir de su georeferencia*

### Superficie

#### 1- Recuperación de informacion faltante en la variable de referencia aplicando regex a la columna descripción

In [ ]:
#se aplica regex a la columna descripcion, buscando las palabras metros, mts y m2 precedido de digitos. 
serie_descripcion = data_limpia["description"]

pattern_sup_m2 = "(?P<sup>\d{2,}(,|.\d+)?)\s*(m2|metros|mts)"
pattern_sup_m2_regex = re.compile(pattern_sup_m2,  re.IGNORECASE)

sup_match = serie_descripcion.apply(lambda x: x if x is np.NaN else pattern_sup_m2_regex.search(x))
mask_sup_match_notnull= sup_match.notnull()

data_limpia.loc[mask_sup_match_notnull, "Sup_m2_Clean"] = sup_match[mask_sup_match_notnull].apply(lambda x: x.group("sup"))

In [ ]:
# Al analizar la información se detecta errores de formato, no posibles de salvaguardar usando los metodos replace o to_numeric, por lo cual se decide aplicar nuevamente regex para sacar la partes enteras de los numeros, lo cual nos permitira poder usar esta informacion en estaditicas mas adelante.
serie_sup_m2_clean=data_limpia["Sup_m2_Clean"]

pattern_sup_m2_dos="(?P<supdos>\d{2,})"
pattern_sup_m2_regex_dos=re.compile(pattern_sup_m2_dos)
sup_match_dos = serie_sup_m2_clean.apply(lambda x: x if x is np.NaN else pattern_sup_m2_regex_dos.search(x))
mask_sup_match_notnull_dos= sup_match_dos.notnull()
data_limpia.loc[mask_sup_match_notnull_dos, "Sup_m2_Clean_dos"] = sup_match_dos[mask_sup_match_notnull_dos].apply(lambda x: x.group("supdos"))


In [ ]:
#se verifica información obtenida con regex
data_limpia[["surface_total_in_m2", "surface_covered_in_m2", "Sup_m2_Clean_dos"]].sample(20)

In [ ]:
#se revisan datos que son mayores a 10000 m2 y que impactarian en el modelo al reemplazar la superficie total.
data_limpia["Sup_m2_Clean_dos"]=pd.to_numeric(data_limpia["Sup_m2_Clean_dos"])
mask_consistencia_regex= (data_limpia["Sup_m2_Clean_dos"] > 10000) & (data_limpia["surface_total_in_m2"].isnull())
consistencia_regex=data_limpia[mask_consistencia_regex]
consistencia_regex[["surface_total_in_m2", "surface_covered_in_m2", "Sup_m2_Clean_dos"]]

In [ ]:
consistencia_regex.shape

In [ ]:
#se decide eliminar los valores analizados por ser estos incoherentes, e impactarian como outliers en los analisis posteriores.
data_limpia["Sup_m2_Clean_tres"]=data_limpia["Sup_m2_Clean_dos"].copy()

data_limpia.loc[mask_consistencia_regex,"Sup_m2_Clean_tres"] = 0

Durante la imputación se podra corroborar el aporte de la información recuperada

#### 2- Revisión la coherencia entre las columnas Superficie total y superficie cubierta.

In [ ]:
#se verifican valores de superficie total vs superficie cubierta, entendiendo que la primera siempre debe ser mayor que la segunda.
mask_consistencia_sup = data_limpia["surface_total_in_m2"] < data_limpia["surface_covered_in_m2"]
data_limpia[mask_consistencia_sup].shape

Se detectan 1106 valores cuya superficie total es menor que la superficie cubierta, siendo esto inconsistente.

Se revisa el contenido de las columnas comparandolas entre si, ademas se la compara con la informacion obtenida en descripcion para comprobar coherencia de los datos

In [ ]:
data_consistencia_sup = data_limpia[mask_consistencia_sup]
data_consistencia_sup[["surface_total_in_m2","surface_covered_in_m2","Sup_m2_Clean_dos", "property_type"]].sample(20)

In [ ]:
#se analiza la consistencia de los datos para valores de superficie mayores a 1000m2
mascara_coherencia_sup = data_limpia["surface_covered_in_m2"] >= 1000
data_coherencia_sup = data_limpia [mascara_coherencia_sup]
data_coherencia_sup [["surface_total_in_m2","surface_covered_in_m2","Sup_m2_Clean_dos", "rooms","property_type"]]

In [ ]:
data_coherencia_sup.shape

Para valores mayores a 1000m2, 39 filas, los valores en la columna superficie total son mas coherente que los datos de superficie cubierta, motivo por el cual en este caso no se modificará la variable

In [ ]:
# se hace mismo analisis par superficies menores a 1000m2
mascara_coherencia_sup_menor = data_limpia["surface_covered_in_m2"] <1000
data_coherencia_sup_menor = data_consistencia_sup [mascara_coherencia_sup_menor]
data_coherencia_sup_menor [["surface_total_in_m2","surface_covered_in_m2","Sup_m2_Clean_dos", "rooms","property_type"]].sample(10)

Para superficies cubiertas menores a 1000m2 la columna superficie cubierta posee valores mas razonales, valores que podrán luego imputarse para obtener un modelo mas preciso.

#### 3- Detección de outliers de la variable Superficie.

In [ ]:
data_limpia.boxplot(column= "surface_total_in_m2", by="property_type") 
plt.show()

In [ ]:
outlier_value = 10000
mask_consistencia_outlier_sup= data_limpia["surface_total_in_m2"] >= outlier_value
data_limpia[mask_consistencia_outlier_sup].shape

Se analiza el contenido de las columnas para corroborar coherencia.

In [ ]:
data_consistencia_outliers = data_limpia[mask_consistencia_outlier_sup]
data_consistencia_outliers[["surface_total_in_m2","surface_covered_in_m2"]].sample(10)

##### Del análisis de las columnas donde existen outliers se detecta que los valores que se encuentran en la columna en superficie cubierta son mas coherentes que los de la columna superficie total, por lo cual, se decide para estas 87 columnas imputar el valor superficie cubierta como superficie total y revisar nuevamente outliers. Ver en apartado *"IMPUTACION"*

##### 4- Se supone que para PH, apartments y store la superficie cubierta es la misma que la superficie total, *se descarta house entendiendo que estas pueden contar con "patio". con estos datos se reemplazan los valores faltantes

# Joni aca iria el analisis gruopby por tipo de propiedad como son las superficies y sus desvios para ver si se cumple la hipotesis

### Precio

### 1-Extraer información con regex de columna descripción

In [ ]:
#generamos una copia del archivo para no modificar archivo base
data_limpia= data.copy()

In [ ]:
#se pretende obtener informacion faltante en la columna precio, aplicando regex a la columna descripción, suponiendo que los valores de precios que nos interesan son en dolares (usd, u$d) seguido de varios digitos.
serie_descripcion = data_limpia["description"]

pattern_usd = "(?P<usd>(usd|u[$]d)\s*?(\d{2,}(,|.\d+)?))"
pattern_usd_regex = re.compile(pattern_usd,  re.IGNORECASE)

usd_match = serie_descripcion.apply(lambda x: x if x is np.NaN else pattern_usd_regex.search(x))

mask_usd_match_notnull= usd_match.notnull()

data_limpia.loc[mask_usd_match_notnull, "usd_clean"] = usd_match[mask_usd_match_notnull].apply(lambda x: x.group(0))

In [ ]:
#la información obtenida posee caracteres no deseables por lo cual se vuelve a aplicar regex para poder dejar datos tipo float.
serie_usd_clean=data_limpia["usd_clean"]
pattern_usd_dos="(?P<usddos>(\d{2,}(,|.\d+)?))"
pattern_usd_regex_dos=re.compile(pattern_usd_dos)
usd_match_dos = serie_usd_clean.apply(lambda x: x if x is np.NaN else pattern_usd_regex_dos.search(x))
mask_usd_match_notnull_dos= usd_match_dos.notnull()
data_limpia.loc[mask_usd_match_notnull_dos, "usd_clean_dos"] = usd_match_dos[mask_usd_match_notnull_dos].apply(lambda x: x.group("usddos"))

### 2-comprobar si las columnas precio ARS y USD pueden matchear

In [ ]:
#se corrobora primero la cantidad de datos nullos de ambas columnas
#ademas se genera una mascara para corroborar si esos datos nulos coinciden, de manera tal que si lo hacen no se podrian matchear columnas.

mask_ARS_empty=data_limpia["price_aprox_local_currency"].isnull()
mask_ARS_empty.value_counts()

In [ ]:
mask_USD_empty=data_limpia["price_aprox_usd"].isnull()
mask_USD_empty.value_counts()

In [ ]:
#se aplica mascara para ambas condiciones antes presentadas.

mask_precio_empty=mask_ARS_empty&mask_USD_empty
data_limpia[mask_precio_empty].shape

##### coinciden la cantidad de filas vacias en cada campo, por lo cual no es posible completar completar datos vacios de USD con ARS sacados de la columna "price_aprox_local_currency"

### 3-Determinar la variacion precio con rooms y superficie

calculamos la correlacion entre variables con un grafico pairgrid, que relaciona las variables por pares

In [ ]:
#se cambian el tipo de datos a las variables numeros y se genera un nuevo dataframe solo con las variables de interes
df_surf_float = data_limpia["surface_total_in_m2"].astype(float) 
df_room_float = data_limpia['rooms'].astype(float) 
df_price_float = data_limpia['price_aprox_usd'].astype(float) 
df_tipo = data_limpia['property_type']

In [ ]:
data_tipo_usd_sup_room=pd.DataFrame((df_tipo,df_surf_float,df_room_float,df_price_float))
data_tipo_usd_sup_room_T=data_tipo_usd_sup_room.T
data_tipo_usd_sup_room_T.sample(10)

In [ ]:
g = sns.PairGrid(data_tipo_usd_sup_room_T, vars=['property_type','surface_total_in_m2','rooms','price_aprox_usd'],hue='property_type', palette='deep')
g.map(plt.scatter, alpha=0.6)
g.add_legend()

Se puede ver una relacion entre el tipo de departamento  con el precio y la superficie total, no asi con la cantidad de habitaciones

### 4- Se pretende determinar la relación entre tipo de propiedades y sus precios. 

In [ ]:
#para confirmar lo visto en el grafico agrupamos el precio por tipo de propiedad para determinar si el precio varia mucho con el tipo de propiedad
data_agrupada_propiedaes= data_limpia.groupby('property_type')
data_agrupada_propiedaes[['property_type','price_aprox_usd']].describe()


# Joni si queres revisa la conclusion de estos casos

# Minar los datos

### Dar formato, limpiar, homogeneizar y filtrar los datos

Decisiones (a tomar):
> Ubicación
> ___
- en cuanto a ubicación quedarse con una columna geometry y el municipio (en el caso de Capital Federal, Comuna == municipio)
  - En caso de geometry, usar lat-lon o geonames_id? porque geonames_id toma el punto medio del municipio (es decir, más inexacto) y lat-lon toma el punto exacto de la propiedad (es decir, más exacto) pero lat-lon tiene muchos valores faltantes.
  - Una opción sería tomar la std media de lat-lon y geonames_id y modificar aleatoriamente geoname para que no quede un punto sobrerepresentado. (comencé a implementar como prueba de concepto pero no lo terminé)
  - Se podría hacer algo similar con los 18640 datos faltantes de geonames_id.
  - En el caso de municipio, habría que imputar unos 29842 valores que utilizan el barrio en vez del municipio ( se ve en places_data_4_elements). 
  - De esta manera en cuanto a municipio sólo perdemos 4780 filas que sólo tienen state_name y no municipio ni ciudad ni es posible imputarlos desde su georeferencia.
> Superficie
> ___
- Queda definar los casos donde superficie cubierta es mayor que superficie total y si es posible imputar la superficie total a partir de la cubierta tomando la cubierta como la total a partir del promedio de superficie total por superficie cubierta.
- Ver qué hacer con los outliers 
- Revisar celda 29 de superficie_limpia_columnas
- Celda 34, discriminar casas y agregar superficie total a partir de la cubierta más un porcentaje de jardín (Jonathan)


> Precio
>- EVALUAR ELIMINAR LOS VALORES Q NO SON PESOS ARGENTINOS PARA PODER HACER UNA CONVERSION DE TODOS LOS VALORES A USD - PENDIENTE

## Imputaciones ubicación

In [ ]:
# Eliminamos las filas que no tienen municipio
data_clean_location = data.drop(index=places_data_2_elements_geo.index)
print(data_clean_location.shape)
data_clean_location.head()

In [ ]:
# Corroboramos que eliminamos nuestro outlier de Colombia
data_clean_location[data_clean_location['lat'] > 0]

In [ ]:
# Reemplazamos ciudades por municipios
data_clean_location.loc[places_data_4_elements_with_mun_as_place_name.index, 'place_name'] = places_data_4_elements_with_mun_as_place_name['place_name']
print(data_clean_location.shape)
data_clean_location.head()

In [ ]:
# Elimino columnas con información duplicadas o innecesaria
unnecesary_columns = ['country_name', 'place_with_parent_names']
data_clean_location = data_clean_location.drop(columns=unnecesary_columns)
print(data_clean_location.shape) 
data_clean_location.head()

In [ ]:
# reutilizamos geo_location_data para obtener lat_geoname y lon_geoname
print(geo_location_data.shape)
geo_location_data.head(3)

In [ ]:
# agregamos lat_geoname y lon_geoname a data_clean_location
data_clean_location['lat_geoname'] = geo_location_data['lat_geoname']
data_clean_location['lon_geoname'] = geo_location_data['lon_geoname']
print(data_clean_location.shape)
data_clean_location.head(3)

In [ ]:
# limpiamos los datos que no tienen georeferencia
data_clean_location.dropna(subset=["geonames_id", 'lat_geoname', 'lon_geoname'], inplace=True)
print(data_clean_location.shape)
data_clean_location.head()

In [ ]:
# Corroboramos la cantidad de nulls en lat y lon
data_clean_location[geolocation + ['lat_geoname', 'lon_geoname']].isna().sum() 

In [ ]:
# Veamos la la media de las lat y lon de la tabla original y los ingresados por geonames_id
place_name_std = data_clean_location.groupby('place_name').agg({'lat':[ 'mean', 'std', 'count'], 'lon': ['mean', 'std',  'count'], 'lat_geoname':[ 'mean', 'std', 'count'], 'lon_geoname': ['mean', 'std', 'count']})
place_name_std.head(3)

In [ ]:
place_name_std.columns = ['lat_mean', 'lat_std', 'lat_count', 'lon_mean', 'lon_std', 'lon_count', 'lat_geoname_mean', 'lat_geoname_std','lat_geoname_count', 'lon_geoname_mean', 'lon_geoname_std', 'lon_geoname_count'] 
place_name_std.head(3)

In [ ]:
### para comparar eventualmente con los valores de referencia

# Por cada place_name, en los casos que no tenemos lat y lon, imputamos la media de lat y lon 
# de geonames_id variada aleatoria con una desviación estándar de cada lugar.
# for index, row in data_clean_location.iterrows():
    #if np.isnan(row['lat']) and np.isnan(row['lon']) and \
    # if place_name_std['lat_geoname_count'][row['place_name']] > 10  and \
    # place_name_std['lon_geoname_count'][row['place_name']] > 10 :
        # data_clean_location.loc[index, 'lat_inferred_for_testing'] = np.random.randn() * place_name_std['lat_std'][row['place_name']] * (1 if np.random.random() < 0.5 else -1 ) + place_name_std['lat_geoname_mean'][row['place_name']]
       #  data_clean_location.loc[index, 'lon_inferred_for_testing'] = np.random.randn() * place_name_std['lon_std'][row['place_name']] * (1 if np.random.random() < 0.5 else -1 ) + place_name_std['lon_geoname_mean'][row['place_name']]

In [ ]:
# Por cada place_name, en los casos que no tenemos lat y lon, imputamos la media de lat y lon 
# de geonames_id variada aleatoria con una desviación estándar de cada lugar.
for index, row in data_clean_location.iterrows():
    if np.isnan(row['lat']) and np.isnan(row['lon']) and \
    place_name_std['lat_geoname_count'][row['place_name']] > 10  and \
    place_name_std['lon_geoname_count'][row['place_name']] > 10 :
        data_clean_location.loc[index, 'lat_inferred'] = np.random.randn() * place_name_std['lat_std'][row['place_name']] * (1 if np.random.random() < 0.5 else -1 ) + place_name_std['lat_geoname_mean'][row['place_name']]
        data_clean_location.loc[index, 'lon_inferred'] = np.random.randn() * place_name_std['lon_std'][row['place_name']] * (1 if np.random.random() < 0.5 else -1 ) + place_name_std['lon_geoname_mean'][row['place_name']]

In [ ]:
# Ubicamos los puntos en el mapa para ver si se distribuyen de forma uniforme.

# Estos df ya fueron creado y están puestos como referencia
# geo_location_data_gdf = gpd.GeoDataFrame(geo_location_data, geometry=gpd.points_from_xy(geo_location_data.lon, geo_location_data.lat))
# geo_location_data_gdf_capital = geo_location_data_gdf[data['state_name']=='Capital Federal']
data_clean_location_capital = data_clean_location[data_clean_location['state_name']=='Capital Federal']

geo_location_data_gdf_capital_geonames_norm = gpd.GeoDataFrame(data_clean_location_capital, geometry=gpd.points_from_xy(data_clean_location_capital.lon_inferred, data_clean_location_capital.lat_inferred))
df_barrios_capital = pd.read_csv('./data/barrios.csv', sep=',', encoding='latin-1')


df_barrios_capital["WKT"] = df_barrios_capital["WKT"].apply(shapely.wkt.loads) 
df_barrios_capital = gpd.GeoDataFrame(df_barrios_capital, geometry='WKT')

fig, ax = plt.subplots(figsize=(10,20))

geo_location_data_gdf_capital.plot(ax=ax, markersize=10, color='red', alpha=0.1) 
geo_location_data_gdf_geonames.plot(ax=ax, markersize=10, color='blue', alpha=0.1)
geo_location_data_gdf_capital_geonames_norm.plot(ax=ax, markersize=10, color='green', alpha=0.1)
df_barrios_capital.plot(ax=ax, alpha=0.4, color='grey', edgecolor='black')
plt.xlim(-58.55,-58.350)
plt.ylim(-34.705,-34.525) 
plt.legend(['lat-lon original', 'punto geonames_id', 'lat-lon inferida a partir de geonames'], loc='upper left')
plt.show()

In [ ]:
# Vemos las propiedades en el mapa de Argentina
argentina = world[world['name'] == 'Argentina']
geo_location_data_gdf_norm = gpd.GeoDataFrame(data_clean_location, geometry=gpd.points_from_xy(data_clean_location.lon_inferred, data_clean_location.lat_inferred)) 
fig, ax = plt.subplots(figsize=(10,10))
argentina.plot(ax=ax, color='grey', edgecolor='black')
data_clean_location_gdf = gpd.GeoDataFrame(geo_location_data_gdf, geometry='geometry')
data_clean_location_gdf.plot( markersize=1, color='red', alpha=0.1,ax=ax)
geo_location_data_gdf_geonames.plot(ax=ax, markersize=1, color='blue', alpha=0.1)
geo_location_data_gdf_norm.plot( markersize=1, color='green', alpha=0.1,ax=ax)
plt.legend(['lat-lon original', 'punto geonames_id', 'lat-lon inferida a partir de geonames'], loc='upper left')
plt.xlim(-72.5,-52.5)
plt.ylim(-56,-20)
plt.show()


> Conclusión: 
> - Imputar place_name a partir de la información recolectada en places_with_parent_names es bastante preciso.
> - Imputar lat y lon a partir de geonames_id es bastante más complicado y habría que ver cómo interactúa con el modelo. Si bien consideramos que vale la pena probarlo, sin mayor información no lo consideramos prudente para agregarlo al dataset
> ___

In [ ]:
data_clean_location

In [ ]:
data_clean_location = data_clean_location.drop(columns=['geonames_id', 'lat', 'lon','lat-lon', "lat_geoname", "lon_geoname", "lat_inferred", "lon_inferred"])


In [ ]:
print(data_clean_location.shape)
data_clean_location.head()

In [ ]:
data_limpia["precio_usd_limpio"]=data_limpia["price_aprox_usd"]
data_limpia["precio_usd_limpio"].fillna(data_limpia["usd_clean_dos"],inplace=True)
data_limpia["precio_usd_limpio"].isnull().sum()

## Imputaciones a variable Superficie

#### 1-se adicionan  la nueva columna "sup_m2_total_limpia" los datos obtenidos de la columna descripcion.

In [ ]:
#recordamos cuantos datos vacíos tiene la columna de interés.
data_limpia["surface_total_in_m2"].isnull().sum()

In [ ]:
data_limpia["sup_m2_total_limpia"] = data_limpia["surface_total_in_m2"].copy()
data_limpia["sup_m2_total_limpia"].fillna(data_limpia["Sup_m2_Clean_tres"],inplace=True)

In [ ]:
print(data_limpia["sup_m2_total_limpia"].isnull().sum())
(1-(data_limpia["sup_m2_total_limpia"].isnull().sum()/data_limpia["surface_total_in_m2"].isnull().sum()))*100

#luego de la imputación de los datos obtenidos por regex la columna disminuye el 31% sus valores nulos.

#### 2- Del analisis de consistencia de datos entre columnas de superficie total vs cubierta, se decide imputar los valores de superficie cubierta a la columna de superficie total, para valores de superficie total menor que superficie cubierta y donde superficie cubierta es menor a 1000m2

In [ ]:
data_limpia.loc[mascara_coherencia_sup_menor,"sup_m2_total_limpia"] = data_limpia["surface_covered_in_m2"]

In [ ]:
print(data_limpia["sup_m2_total_limpia"].isnull().sum())
(1-(data_limpia["sup_m2_total_limpia"].isnull().sum()/data_limpia["surface_total_in_m2"].isnull().sum()))*100

obteniendo una redución del 78.9% de la información faltante

##### 3- Del análisis de las columnas donde existen outliers se detecta que los valores en superficie cubierta son mas coherentes por lo cual, se decide para estas 87 columnas imputar el valor superficie cubierta como superficie total

In [ ]:
data_limpia.loc[mask_consistencia_outlier_sup,"sup_m2_total_limpia"] = data_limpia["surface_covered_in_m2"]

In [ ]:
data_limpia["sup_m2_total_limpia"]=pd.to_numeric(data_limpia["sup_m2_total_limpia"])

In [ ]:
data_limpia.boxplot("sup_m2_total_limpia", by="property_type") 
plt.show()

In [ ]:
outlier_value = 10000
mask_consistencia_outlier_sup_dos= data_limpia["sup_m2_total_limpia"] >= outlier_value
data[mask_consistencia_outlier_sup_dos].shape

In [ ]:
consistencia_outlier_sup_dos= data_limpia[mask_consistencia_outlier_sup_dos]
consistencia_outlier_sup_dos[["surface_total_in_m2","surface_covered_in_m2","sup_m2_total_limpia", "property_type","rooms","Sup_m2_Clean_tres"]]

##### 4- Se supone que para PH, apartments y store la superficie cubierta es la misma que la superficie total, *se descarta house entendiendo que estas pueden contar con "patio". con estos datos se reemplazan los valores faltantes 

# JONI ACA VA EL CODIGO QUE ARREGLATE VOS
lo q esta aca abajo es lo que hice yo, q considera todo, o sea no excluye a "house"

In [ ]:
data_key_not_null_mask = np.logical_and(data_limpia.property_type.notnull(), data.surface_covered_in_m2.notnull())
data_key_not_null = data_limpia.loc[data_key_not_null_mask, :]
data_key_not_null.shape

In [ ]:
data_key_not_null_grouped_tipo_sup= data_key_not_null.groupby(["property_type", "surface_covered_in_m2"])
data_fillna= data_key_not_null_grouped_tipo_sup["surface_total_in_m2"].transform(lambda grp: grp.fillna(grp.mean()))
data_limpia["sup_m2_total_limpia"].fillna(data_fillna,inplace=True)

In [ ]:
data_limpia["sup_m2_total_limpia"].isnull().sum()

## Imputaciones a variable Precio

#### 1- Imputación de información obtenida a través de regex

In [ ]:
(1-data_limpia["precio_usd_limpio"].isnull().sum()/data["price_aprox_usd"].isnull().sum())*100

Se recuperó el 1.2% de la informacion de precios en dolares del campo descripcion. La recuperacion de datos no es significativa. 

####  2- Imputación a columna precios el valor del precio promedio por tipo de propiedad

In [ ]:
#Se toma el valor medio del precio de propiedad por tipo de propiedad y se completan faltantes.
data_key_not_null_mask_precio = np.logical_and(data_limpia.property_type.notnull(), data_limpia.price_aprox_usd.notnull())
data_key_not_null_precio = data_limpia.loc[data_key_not_null_mask_precio, :]

data_key_not_null_grouped_tipo_precio= data_key_not_null_precio.groupby(["property_type", 'price_aprox_usd'])


data_fillna_precio= data_key_not_null_grouped_tipo_precio['price_aprox_usd'].transform(lambda grp: grp.fillna(grp.mean()))
data_fillna_precio

data_limpia["precio_usd_limpio"].fillna(data_fillna_precio,inplace=True)

In [ ]:
(1-data_limpia["precio_usd_limpio"].isnull().sum()/data["price_aprox_usd"].isnull().sum())*100

Se recupera el 67% de la informacion faltante 

In [ ]:
data_limpia["precio_usd_limpio"].isnull().sum()/data_limpia.shape[0]*100

queda el 5.5% de la columna con faltantes.

# TODO: MEJORAR EL RANDOM DE LA IMPUTACIÓN DE GEONAMES_ID ALGUNOS QUEDAN MUY LEJOS. REPENSAR

In [ ]:
# Por cada place_name, en los casos que no tenemos lat y lon, imputamos la media de lat y lon 
# de geonames_id variada aleatoria con una desviación estándar de cada lugar.


for index, row in data_clean_location.iterrows():
    if np.isnan(row['lat']) and np.isnan(row['lon']):
        data_clean_location.loc[index, 'lat'] = np.random.randn() * place_name_std['lat_std'][row['place_name']] + place_name_std['lat_mean'][row['place_name']]
        data_clean_location.loc[index, 'lon'] = np.random.randn() * place_name_std['lon_std'][row['place_name']] + place_name_std['lon_mean'][row['place_name']]

In [ ]:
# Corroboramos la cantidad de nulls en lat y lon
data_clean_location[geolocation].isna().sum()  

In [ ]:
# Ubicamos los puntos en el mapa para ver si se distribuyen de forma uniforme.

# Estos df ya fueron creado y están puestos como referencia
# geo_location_data_gdf = gpd.GeoDataFrame(geo_location_data, geometry=gpd.points_from_xy(geo_location_data.lon, geo_location_data.lat))
# geo_location_data_gdf_capital = geo_location_data_gdf[data['state_name']=='Capital Federal']


geo_location_data_gdf_capital_geonames_norm = gpd.GeoDataFrame(data_clean_location, geometry=gpd.points_from_xy(data_clean_location.lon, data_clean_location.lat))
geo_location_data_gdf_capital['geometry']
df_barrios_capital = pd.read_csv('./data/barrios.csv', sep=',', encoding='latin-1')


df_barrios_capital["WKT"] = df_barrios_capital["WKT"].apply(shapely.wkt.loads) 
df_barrios_capital = gpd.GeoDataFrame(df_barrios_capital, geometry='WKT')

geo_location_data_gdf_capital['geometry']
fig, ax = plt.subplots(figsize=(10,20))

geo_location_data_gdf_capital.plot(ax=ax, markersize=1, color='red', alpha=0.1) 
geo_location_data_gdf_capital_geonames_norm.plot(ax=ax, markersize=1, color='blue', alpha=0.1)
df_barrios_capital.plot(ax=ax, alpha=0.4, color='grey', edgecolor='black')
plt.xlim(-58.55,-58.350)
plt.ylim(-34.705,-34.525) 
plt.show()

In [ ]:
# Rearmamos la columna geometry con los nuevos valores de lat y lon
data_clean_location['geometry'] = gpd.points_from_xy(data_clean_location.lon, data_clean_location.lat)
# Limpiamos las columnas redundantes
data_clean_location = data_clean_location.drop(columns=['lat', 'lon', 'lat_geoname', 'lon_geoname', 'geonames_id', 'lat-lon',])
print(data_clean_location.shape)
data_clean_location.head()


In [ ]:
# Vemos las propiedades en el mapa de Argentina
argentina = world[world['name'] == 'Argentina'] 
fig, ax = plt.subplots(figsize=(10,10))
argentina.plot(ax=ax, color='grey', edgecolor='black')
data_clean_location_gdf = gpd.GeoDataFrame(data_clean_location, geometry='geometry')
data_clean_location_gdf.plot(figsize=(10,10), markersize=1, color='red', alpha=0.1,ax=ax)
plt.show()


# Refinar los datos

# Exportar el nuevo dataset 